![](imgs/logo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.


## Podstawy Spark SQL - UDF

Podobnie jak w Hive czy wielu bazach danych, Spark SQL ma możliwość definiowania funkcji użytkownika, ang. User Defined Functions (UDF). Funkcje te biorą wartość z kolumny i przekształcają ją w inną wartość. 

In [ ]:
import pyspark
import pyspark.sql.functions as func
import pyspark.sql.types as types
sc = pyspark.SparkContext(appName='udf')
sqlContext = pyspark.sql.SQLContext(sc)

Wygenerujmy najpierw dane:

In [ ]:
import numpy as np
df = sqlContext.createDataFrame([pyspark.sql.Row(kolumna=int(i)) for i in np.random.randint(0, 100, 100)])
df.printSchema()
df.show()

Do stworzenia funkcji używamy normalnej referencji do funkcji Python lub lambdy. Przykładowo, chcemy funkcję która zwróci klasyfikacje wartości:

In [ ]:
def klasyfikuj(wartosc):
    return u'dużo' if wartosc > 50 else u'mało'

Powyższą funkcję należy teraz przekształcić w funkcje która działa na kolumnach. Robimy to używając funkcji `udf`:

In [ ]:
klasyfikuj_udf = func.udf(klasyfikuj)

Tej funkcji można już użyć na kolumnie DataFrame, np możemy dodać kolumnę:

In [ ]:
df.withColumn('ile?', klasyfikuj_udf('kolumna')).show()

Należu uważać z typami; jeżeli chcemy zwrócić inny typ niż tekstowy, należy przekazać tą informację w definicji funkcji UDF.

In [ ]:
oblicz_udf = func.udf(lambda v: v + 123, returnType=types.IntegerType())

In [ ]:
df.withColumn('oblicz', oblicz_udf('kolumna')).show()

Aby wykorzystać funckje w zapytaniach SQL należy ją zarejestrować nieco inaczej, mianowicie:

In [ ]:
sqlContext.udf.register('klasyfikuj', klasyfikuj, returnType=types.StringType())

In [ ]:
df.registerTempTable('df')
sqlContext.sql('SELECT kolumna, klasyfikuj(kolumna) AS `ile?` FROM df').show()

## Dostęp do JVM

Spark używa [py4j](https://www.py4j.org/) aby wykonywać komendy na JVM. Dostęp do klas mamy w PySpark nieco ułatwiony, mianowicie używamy `_jvm` ze `SparkContext` podając pełną nazwę klasy z pakietem:

In [ ]:
s = sc._jvm.java.lang.String('tekst')
print(s, type(s))

Jak widać, py4j konwertuje typy. Możemy też użyć bardziej skomplikowanych obiektów:

In [ ]:
tokenizer = sc._jvm.java.util.StringTokenizer('Ala ma kota!')
print(tokenizer, type(tokenizer))

In [ ]:
while(tokenizer.hasMoreTokens()):
    print(tokenizer.nextToken())

Lub prościej:

In [ ]:
p = sc._jvm.java.util.regex.Pattern.compile('\\s')
print(p, type(p))
l = p.split('Ala ma kota!')
print(l, type(l))
list(l)

Oczywiście lepiej powyższą funkcjonalność wykorzystać do funkcji zwracających RDD lub DataFrame, które można użyć bezpośrednio w Pythonie.